In [31]:
import os
from dotenv import load_dotenv
from langchain_neo4j import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

import vansy_f8a_genai_test.report_a03_rag_and_reasoning_frameworks_tutorial_prompt as prompt_hub

In [32]:
load_dotenv()

True

In [33]:
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

/var/folders/lg/yb401ry14w19vfs73n7y2rtw0000gn/T/ipykernel_74060/88119791.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


### Hospital Cypher Chain

In [34]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

graph.refresh_schema()

In [35]:
cypher_generation_template = prompt_hub.CYPHER_GENERATION_TEMPLATE
qa_generation_template = prompt_hub.QA_GENERATION_TEMPLATE

In [36]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"], template=cypher_generation_template
)

qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)


In [37]:
hospital_cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    qa_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    graph=graph,
    verbose=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    validate_cypher=True,
    top_k=100,
    allow_dangerous_requests=True
)

### ReAct Agent

In [38]:
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_openai import ChatOpenAI
from langchain import hub

In [39]:
# 1. Wrap your chain in the Tool class
hospital_tool = Tool(
    name="HospitalGraphQA",
    # The 'func' is the chain's invoke method. We use a lambda to map the
    # string input from the agent to the dictionary input the chain expects.
    func=lambda q: hospital_cypher_chain.invoke({"query": q}),
    description="""Useful for answering questions about patients,
        physicians, hospitals, insurance payers, patient review
        statistics, and hospital visit details. Use the entire prompt as
        input to the tool. For instance, if the prompt is 'How many visits
        have there been?", the input should be "How many visits have
        there been?'"""
)

In [40]:
# 2. Define the tools for agent can use
tools = [hospital_tool]

In [41]:
# 3. Set up the ReAct Agent
# Pull the standard ReAct prompt
prompt = hub.pull("hwchase17/react")

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [42]:
prompt.dict()

{'name': None,
 'input_variables': ['agent_scratchpad', 'input', 'tool_names', 'tools'],
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {},
 'metadata': {'lc_hub_owner': 'hwchase17',
  'lc_hub_repo': 'react',
  'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'},
 'tags': None,
 'template': 'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}',
 'template_format': 'f-string',
 'validate_t

In [43]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [26]:
# Create the agent
agent = create_react_agent(agent_llm, tools, prompt)

In [44]:
# Create the agent executor to run the agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [45]:
# --- 4. The Interactive Chat Loop ---
print("Chat with the Hospital AI Agent! Type 'exit' to quit.")
while True:
    try:
        user_question = input("You: ")
        print(f"User: {user_question}")
        if user_question.lower() in ["exit", "quit"]:
            print("\nAgent: Goodbye!")
            break
        
        # Invoke the agent executor with the user's question
        response = agent_executor.invoke({"input": user_question})
        
        # Print the final answer
        print(f"\nAgent: {response['output']}")

    except Exception as e:
        print(f"An error occurred: {e}")

Chat with the Hospital AI Agent! Type 'exit' to quit.
User: How many patients were treated by Denise Chavez?


> Entering new AgentExecutor chain...
I need to find out how many patients were treated by Denise Chavez. This requires querying the relevant data about physicians and their patient statistics. 
Action: HospitalGraphQA
Action Input: How many patients were treated by Denise Chavez?

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (phy:Physician {name: 'Denise Chavez'})-[:TREATS]->(v:Visit)<-[:HAS]-(p:Patient)
RETURN COUNT(p) AS patient_count
Full Context:
[{'patient_count': 16}]

> Finished chain.
{'query': 'How many patients were treated by Denise Chavez?', 'result': 'Denise Chavez treated a total of 16 patients.'}I now know the final answer.  
Final Answer: Denise Chavez treated a total of 16 patients.

> Finished chain.

Agent: Denise Chavez treated a total of 16 patients.
User: How many hospital in Florida?


> Entering new AgentExecutor chain...
I need t